In [ ]:
!pip install transformers torch scikit-learn --quiet

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from sklearn.metrics import classification_report
from tqdm import tqdm
import os

In [ ]:
from data_loaders import FactCheckDataset, create_dataloaders

In [ ]:
model_name = "vinai/phobert-base"
train_path = "../data/vihallu-train.csv"
test_path  = "../data/vihallu-public-test.csv"

In [ ]:
batch_size = 8
max_len = 128
epochs = 10
lr = 2e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

train_loader, test_loader, label2id = create_dataloaders(
    train_path, test_path, tokenizer,
    batch_size=batch_size,
    max_len=max_len
)

id2label = {v:k for k,v in label2id.items()}
print("Label mapping:", label2id)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)
model.to(device)

In [ ]:
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
for epoch in range(epochs):
    print(f"\n===== Epoch {epoch+1}/{epochs} =====")

    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc="Training", leave=True)
    for batch in loop:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Average training loss for epoch {epoch+1}: {avg_loss:.4f}")

In [ ]:
model.eval()
preds, trues = [], []
with torch.no_grad():
    for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids,
                            attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            preds.extend(predictions.cpu().numpy())
            trues.extend(labels.cpu().numpy())

print("Classification Report:")
print(classification_report(trues, preds, target_names=list(label2id.keys())))

In [ ]:
save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Model saved to {save_dir}")